# Logistic Regression and Boosting Algorithms


<a id="predicting-a-categorical-response"></a>
## Predicting a Single Categorical Response
---



### Installing stuff

In [ ]:
!pip install --upgrade textblob spacy gensim swifter

In [ ]:
!python -m textblob.download_corpora lite
!python -m spacy download en_core_web_sm

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB         # Naive Bayes
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
import torch
import spacy
import gensim
import warnings
import nltk
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words


In [ ]:
%%writefile get_data.sh
if [ ! -f yelp.csv ]; then
  wget -O yelp.csv https://www.dropbox.com/s/xds4lua69b7okw8/yelp.csv?dl=0
fi

In [ ]:
!bash get_data.sh

In [ ]:
# Read yelp.csv into a DataFrame.
path = './yelp.csv'
yelp = pd.read_csv(path)
# Create a new DataFrame that only contains the 5-star and 1-star reviews.
yelp_best_worst = yelp[(yelp.stars==5) | (yelp.stars==1)]

# Define X and y.
X = yelp_best_worst.text
y = yelp_best_worst.stars

# Split the new DataFrame into training and testing sets.


<a id="using-logistic-regression-for-classification"></a>
## Using Logistic Regression for Classification
---

Logistic regression is a more appropriate method for what we just did with a linear regression. The values output from a linear regression cannot be interpreted as probabilities of class membership since their values can be greater than 1 and less than 0. Logistic regression, on the other hand, ensures that the values output as predictions can be interpreted as probabilities of class membership.

**Import the `LogisticRegression` class from `linear_model` below and fit the same regression model predicting `stars` from `text`.**

In [ ]:
# Fit a logistic regression model and store the class predictions.
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

logreg.fit(X,y)


Of course this simply fails, we need to preprocess the text, convert it into a Tensor format and then and only then we can use models!

### Converting text to vectors

In [ ]:
import re
nltk.download('stopwords')
my_stopwords = nltk.corpus.stopwords.words('english')
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'


def preprocess_text(text, should_join=True):
    text = ' '.join(word.lower() for word in textblob_tokenizer(text))
    text = re.sub(r'http\S+', '', text) # remove http links
    text = re.sub(r'bit.ly/\S+', '', text) # rempve bitly links
    text = text.strip('[link]') # remove [links]
    text = re.sub('['+my_punctuation + ']+', ' ', text) # remove punctuation
    text = re.sub('\s+', ' ', text) #remove double spacing
    text = re.sub(r"[^a-zA-Z.,&!?]+", r" ", text) # only normal characters
    text_token_list = [word for word in text.split(' ')
                            if word not in my_stopwords] # remove stopwords
    text_token_list = [word_rooter(word) if '#' not in word else word
                        for word in text_token_list] # apply word rooter
    text = ' '.join(text_token_list)
    if should_join:
      return ' '.join(gensim.utils.simple_preprocess(text))
    else:
      return gensim.utils.simple_preprocess(text)

In [ ]:
import swifter
X_preprocessed = X.swifter.apply(preprocess_text)

In [ ]:
X_preprocessed[0]

How do we pass from text to numbers? With tokenizers. We will use PyTorch ones!

In [ ]:
def get_maximum_review_length(srs):
    maximum = 0
    for response in srs:
        candidate = len(preprocess_text(response, should_join=False))
        if candidate > maximum:
            maximum = candidate
    return maximum


maximum = get_maximum_review_length(X_preprocessed)

In [ ]:
print(f'The maximum review was {maximum} words long')

In [ ]:
!pip install pytorch-nlp

In [ ]:
import itertools
from torchnlp.encoders import LabelEncoder
list_of_words = list(itertools.chain.from_iterable([preprocess_text(sentence, should_join=False) for sentence in X_preprocessed.values]))
ids_from_words = LabelEncoder(list_of_words, reserved_labels=['UNK'], unknown_index=0, min_occurrences=min_count_words)


In [ ]:
ids_from_words.batch_encode(["breakfast"])

In [ ]:
ids_from_words.decode(ids_from_words.encode("breakfast"))

In [ ]:
def text_from_ids(ids):
  return ids_from_words.batch_decode(ids)

In [ ]:
def ids_from_text(text):
  return ids_from_words.batch_encode(text)

In [ ]:
ids = ids_from_text('Only you can prevent forest fires'.lower().split())
ids

In [ ]:
text_from_ids(ids)


In [ ]:
def pad_sequence_of_tokens(x, maxlen, unk_token='[UNK]'):
  if len(x)<maxlen:
    x.extend([unk_token]*(maxlen-len(x)))
  return x

def get_tensor(x, maximum=maximum):
  padding = (0, maximum-ids_from_text(x).shape[-1])
  return torch.squeeze(F.pad(ids_from_text(x), padding, "constant", 0).to(torch.long))

In [ ]:
import torch.nn.functional as F
def get_ids_tensor(srs):

  processed = srs.swifter.apply(lambda x: pad_sequence_of_tokens(preprocess_text(x, should_join=False), maxlen=maximum))
  result = processed.swifter.apply(get_tensor).to_list()
  return torch.stack(result)


In [ ]:
all_ids = get_ids_tensor(srs=X_preprocessed.reset_index(drop=True))
all_ids

In [ ]:
all_ids.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_ids.numpy(), y, test_size=0.25 ,random_state=99)

### Using Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)



In [ ]:
from sklearn import metrics
y_pred = logreg.predict(X_test)
print((metrics.accuracy_score(y_test, y_pred)))

## Using Boosting Algorithms and other things

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=50, learning_rate=0.5)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=50, learning_rate=0.5)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=50)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

## Multiclass Classification

Just check in the estimators, most support multiclass classification.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
X, y = load_iris(return_X_y=True)
clf = LogisticRegression(random_state=0, multi_class='multinomial').fit(X, y)
clf.predict(X[:2, :])
clf.predict_proba(X[:2, :])
clf.score(X, y)

### **Homework**: Try to perform the stars classification with Logistic Regression but without filtering only for 5 and 1 stars.